In [10]:
# text -> klueberta
# klue_roberta_multimodal

In [11]:
import numpy as np

In [12]:
audio_truth = np.load('data/audio_truth.npy')
text_truth = np.load('data/text_truth.npy')
multi_modal_truth = np.load('data/klue_roberta-base_truth.npy')

In [13]:
audio_pred = np.load('data/audio_pred.npy')
text_pred = np.load('data/text_pred.npy')
multi_modal_pred = np.load('data/klue_roberta-base_pred.npy')

In [14]:
audio_prob = np.load('data/audio_output.npy')
text_prob = np.load('data/text_output.npy')
multi_modal_prob = np.load('data/klue_roberta-base_output.npy')

In [15]:
print("audio")
print("실제값", audio_truth.shape)
print("예측클래스", audio_pred.shape)
print("확률", audio_prob.shape)
print("text")
print("실제값", text_truth.shape)
print("예측클래스", text_pred.shape)
print("확률", text_prob.shape)
print("multi-modal")
print("실제값", multi_modal_truth.shape)
print("예측클래스", multi_modal_pred.shape)
print("확률", multi_modal_prob.shape)

audio
실제값 (3093,)
예측클래스 (3093,)
확률 (3093, 4)
text
실제값 (3093,)
예측클래스 (3093,)
확률 (3093, 4)
multi-modal
실제값 (3093,)
예측클래스 (3093,)
확률 (3093, 4)


# stacking
- lgbm 모델 사용

In [16]:
audio_text_multi_pred = np.array([audio_pred, text_pred, multi_modal_pred])
audio_text_multi_pred.shape

(3, 3093)

In [17]:
#행이 2025개. 열이 2개로 바꿔줘야함 !!!
audio_text_cm_pred = np.transpose(audio_text_multi_pred)
audio_text_cm_pred[:10]

array([[2, 2, 1],
       [2, 2, 1],
       [2, 3, 2],
       [2, 1, 2],
       [2, 2, 1],
       [2, 3, 1],
       [2, 2, 2],
       [2, 0, 1],
       [2, 0, 1],
       [2, 2, 2]])

In [19]:
import lightgbm as lgbm
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

lgbm = LGBMClassifier()
lgbm.fit(audio_text_cm_pred, audio_truth)
lgbm_pred = lgbm.predict(audio_text_cm_pred)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000371 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12
[LightGBM] [Info] Number of data points in the train set: 3093, number of used features: 3
[LightGBM] [Info] Start training from score -1.602350
[LightGBM] [Info] Start training from score -1.277642
[LightGBM] [Info] Start training from score -0.999869
[LightGBM] [Info] Start training from score -1.884164
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [20]:
print("정확도: {0:.4f}".format(accuracy_score(audio_truth, lgbm_pred)))
print("precision_score: {0:.4f}".format(precision_score(audio_truth, lgbm_pred, average="weighted")))
print("recall_score: {0:.4f}".format(recall_score(audio_truth, lgbm_pred, average="weighted")))
print("f1_score: {0:.4f}".format(f1_score(audio_truth, lgbm_pred, average="weighted")))
     

정확도: 0.6133
precision_score: 0.6091
recall_score: 0.6133
f1_score: 0.5978


# soft voting

In [21]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [23]:
#softmax 적용
audio_prob = softmax(audio_prob)
text_prob = softmax(text_prob)
mul_modal_prob = softmax(multi_modal_prob)

In [27]:
audio_text_multi_pred_list = audio_prob + text_prob + multi_modal_prob

In [25]:
labels = [0,1,2,3]

In [28]:
soft_voting_label = []
for i in range(len(audio_truth)):
  label = labels[audio_text_multi_pred_list[i].argmax()]
  soft_voting_label.append(label)

In [29]:
len(soft_voting_label) #3093

3093

In [30]:
print("정확도: {0:.4f}".format(accuracy_score(audio_truth, soft_voting_label)))
print("precision_score: {0:.4f}".format(precision_score(audio_truth, soft_voting_label, average="weighted")))
print("recall_score: {0:.4f}".format(recall_score(audio_truth, soft_voting_label, average="weighted")))
print("f1_score: {0:.4f}".format(f1_score(audio_truth, soft_voting_label, average="weighted")))

정확도: 0.6117
precision_score: 0.6075
recall_score: 0.6117
f1_score: 0.5977


### weighted soft voting

In [31]:
audio_text_multi_pred_2 = 0.1*audio_prob + 0.67*text_prob + 0.23*multi_modal_prob

In [32]:
wsoft_voting_label = []
for i in range(len(audio_truth)):
  label = labels[audio_text_multi_pred_2[i].argmax()]
  wsoft_voting_label.append(label)

In [33]:
len(wsoft_voting_label) # 3093

3093

In [34]:
print("정확도: {0:.4f}".format(accuracy_score(audio_truth, wsoft_voting_label)))
print("precision_score: {0:.4f}".format(precision_score(audio_truth, wsoft_voting_label, average="weighted")))
print("recall_score: {0:.4f}".format(recall_score(audio_truth, wsoft_voting_label, average="weighted")))
print("f1_score: {0:.4f}".format(f1_score(audio_truth, wsoft_voting_label, average="weighted")))

정확도: 0.6117
precision_score: 0.6075
recall_score: 0.6117
f1_score: 0.5977


# hard voting

In [35]:
#가장 많이 나온 결과를 채택하는 코드
def mode(list, weights):
    count = 0
    mode = 0
    for i, x in enumerate(list): 
        if list.count(x) * weights[i] > count:
            count = list.count(x) * weights[i]
            mode = x

    return mode

In [36]:
hard_vot = []
for i in range(len(audio_pred)):
  index =  mode([audio_pred[i], text_pred[i], multi_modal_pred[i]], [0.2, 0.5, 0.3])
  hard_vot.append(index)

In [37]:
print("정확도: {0:.4f}".format(accuracy_score(audio_truth, hard_vot)))
print("precision_score: {0:.4f}".format(precision_score(audio_truth, hard_vot, average="weighted")))
print("recall_score: {0:.4f}".format(recall_score(audio_truth, hard_vot, average="weighted")))
print("f1_score: {0:.4f}".format(f1_score(audio_truth, hard_vot, average="weighted")))

정확도: 0.4093
precision_score: 0.3523
recall_score: 0.4093
f1_score: 0.3559
